In [ ]:
import re
import numpy as np
import pandas as pd
from collections import defaultdict
import csv
from google.colab import drive

In [ ]:
drive.mount('/content/drive')

In [ ]:
!pip install datasets
!pip install transformers
!pip install stanza

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 487.4/487.4 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 15.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 18.2 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.0
    Uninstalling fsspec-2025.3.0:
      Successfully uninstalled fsspec-2025.3.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.0 requires fsspec==2025.3.0, but you have fsspec 2024.12.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which 

In [ ]:
from datasets import load_dataset

ds = load_dataset("Salesforce/wikitext", "wikitext-103-raw-v1")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/733k [00:00<?, ?B/s]

train-00000-of-00002.parquet:   0%|          | 0.00/157M [00:00<?, ?B/s]

train-00001-of-00002.parquet:   0%|          | 0.00/157M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/657k [00:00<?, ?B/s]

Generating test split:   0%|          | 0/4358 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/1801350 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3760 [00:00<?, ? examples/s]

In [ ]:
text = [para["text"] for para in ds['train']]

In [ ]:
text_clean = [sent for sent in text if sent != ""]
text_clean = [re.sub(r'[\t\n]', "", sent, flags=re.MULTILINE) for sent in text_clean]

In [ ]:
len(text_clean)

1165029

In [ ]:
# seperate each paragraph into sentences
text_clean = [re.split(r' \. ', para) for para in text_clean]
text_clean = [para.strip() for sublist in text_clean for para in sublist]

In [ ]:
# remove sentences with only empty strings
text_clean = [sent for sent in text_clean if sent != ""]

In [ ]:
text_clean = [text+"." for text in text_clean]

In [ ]:
len(text_clean)

4089708

In [ ]:
import stanza

nlp = stanza.Pipeline(lang='en', processors='tokenize,mwt,pos,lemma,depparse')

INFO:stanza:Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


INFO:stanza:Downloaded file to /root/stanza_resources/resources.json


INFO:stanza:Loading these models for language: en (English):
| Processor | Package           |
---------------------------------
| tokenize  | combined          |
| mwt       | combined          |
| pos       | combined_charlm   |
| lemma     | combined_nocharlm |
| depparse  | combined_charlm   |

INFO:stanza:Using device: cuda
INFO:stanza:Loading: tokenize
INFO:stanza:Loading: mwt
INFO:stanza:Loading: pos
INFO:stanza:Loading: lemma
INFO:stanza:Loading: depparse
INFO:stanza:Done loading processors!


In [ ]:
verbs = ["agree", "believe", "bet", "consider", "decide", "expect", "feel",
         "figure", "find", "guess", "hear", "hope", "imagine", "know", "mean",
         "notice", "read", "realize", "remember", "say", "see", "show",
         "suppose", "take", "teach", "tell", "thank", "think", "understand",
         "wish", "worry"]
additional_verbs = ["accept", "acknowledge", "add", "affirm", "allege",
                    "announce", "answer", "anticipate", "argue", "assert",
                    "attest", "boast", "brag", "calculate", "caution",
                    "certify", "claim", "comment", "complain", "conceal",
                    "concede", "confess", "confirm", "declare", "deduce",
                    "demonstrate", "deny", "determine", "deny", "determine",
                    "disclose", "doubt", "dream", "emphasize", "establish",
                    "estimate", "explain", "fear", "forget", "gloat", "growl",
                    "guarantee", "hate", "hint", "holler", "hoot", "hypothesize",
                    "ignore", "imply", "indicate","infer", "insist", "intimate",
                    "joke", "learn", "like", "maintain", "mention", "moan",
                    "mumble", "murmur", "muse", "mutter", "note", "observe",
                    "opine", "perceive", "plead", "predict", "presume", "pretend",
                    "proclaim", "promise", "propose", "prove", "reason", "recall",
                    "reckon", "recognize", "recollect", "regret", "reiterate",
                    "remark", "repeat", "reply", "report", "request", "resent",
                    "respond", "reveal", "scream", "sense", "shout", "shriek",
                    "signal", "signify", "speculate", "stammer", "state",
                    "suggest", "suspect", "swear", "testify", "theorize", "trust",
                    "verify", "vow", "wail", "warn", "whine", "whisper", "wonder",
                    "write", "yell"]
all_verbs = verbs + additional_verbs

In [ ]:
other_comp_list = ["who", "what", "where", "why", "how", "whom", "whose", "when",
                   "whether", "if"]

test_sentences = text_clean[80000:100000]
verb_counts=[]
sentences_with_comp = []
freq_dicts = {
        "overall": defaultdict(int),
        "that": defaultdict(int),
        "other": defaultdict(int),
        "none": defaultdict(int),
    }

for test_sentence in test_sentences:
  test = nlp(test_sentence)
  for sentence in test.sentences:
    id_to_token = {token.id: token for token in sentence.words}
    children_dict = {token.id: [] for token in sentence.words}
    for word in sentence.words:
      if word.head > 0:  # Skip root (head=0)
        children_dict[word.head].append(word)

      # count the occurance of the verb
      if word.upos == "VERB" and word.lemma in all_verbs:
        verb_counts.append(word.lemma)
        freq_dicts["overall"][word.lemma] += 1

      # count the number of times the verb is taking a complement clause,
      # regardless of whether it is headed by "that"
      if word.deprel == "ccomp":
        # comp_sentences.append(test_sentence)
        embedded_verb_head_id = word.head
        if embedded_verb_head_id > word.id:
          continue
        matrix_verb_entry = sentence.words[embedded_verb_head_id-1]
        if matrix_verb_entry.lemma in all_verbs:
          # the cc is markecd by "that"
          for child in children_dict[word.id]:
            if child.lemma == "that" and child.deprel == "mark":
              # number of words between the verb and the CC
              matrx_verb_to_cc = child.id - matrix_verb_entry.id - 1

              child_begin_loc = child.start_char
              child_end_loc = child.end_char
              pseudo_that_sentences = test_sentence[:child_begin_loc] + test_sentence[child_end_loc:]
              pseudo_that_sentences = " ".join(pseudo_that_sentences.split())  # Clean up extra spaces.
              comp_type = "that"
              freq_dicts["that"][matrix_verb_entry.lemma] += 1
              break
            # the cc is marked by some other complementizer
            elif child.lemma in other_comp_list:
              # number of words between the verb and the CC
              matrx_verb_to_cc = child.id - matrix_verb_entry.id - 1
              pseudo_that_sentences = test_sentence
              comp_type = "other"
              freq_dicts["other"][matrix_verb_entry.lemma] += 1
              break
            # the cc is not marked by any overt complementizer
            else:
              # number of words between the verb and the CC
              matrx_verb_to_cc = 0
              pseudo_that_sentences = test_sentence
              comp_type = "none"
              freq_dicts["none"][matrix_verb_entry.lemma] += 1
              break
          sentences_with_comp.append({
              "sentence": test_sentence,
              "pseudo_that": pseudo_that_sentences,
              "matrix_verb_to_cc": matrx_verb_to_cc,
              "verb": matrix_verb_entry.lemma,
              "verb_in_text": matrix_verb_entry.text,
              "comp_type": comp_type
          })


sentences_with_comp_df = pd.DataFrame(sentences_with_comp)
sentences_with_comp_df.to_csv("/content/sentences_with_comp_test_8.csv", index=False)

verbs_encountered = set(freq_dicts["overall"].keys()) | set(freq_dicts["that"].keys()) | \
                    set(freq_dicts["other"].keys()) | set(freq_dicts["none"].keys())
freq_data = []
for verb in verbs_encountered:
    freq_data.append({
        "verb": verb,
        "overall": freq_dicts["overall"].get(verb, 0),
        "that": freq_dicts["that"].get(verb, 0),
        "other": freq_dicts["other"].get(verb, 0),
        "none": freq_dicts["none"].get(verb, 0)
    })
df_freq = pd.DataFrame(freq_data)
df_freq.to_csv("/content/verb_freq_verb_test_8.csv", index=False)

print("verb_counts", len(verb_counts))

verb_counts 6578


In [ ]:
sentences_with_comp_df.to_csv("/content/drive/MyDrive/comp_drop/sentences_with_comp_test_8.csv", index=False)
df_freq.to_csv("/content/drive/MyDrive/comp_drop/verb_freq_verb_test_8.csv", header=0)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
